# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from nltk.stem.wordnet import WordNetLemmatizer
import re
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [13]:
# load data from database
engine = create_engine('sqlite:///Disaster_Response.db')
conn=engine.connect()
df = pd.read_sql_table('msg_ctg', conn)
X = df['message']
Y = df.drop(['id','message','genre','original'], axis=1)
Y.head()
X.head()
X.shape, Y.shape

((26216,), (26216, 36))

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    lemmatizer = WordNetLemmatizer()
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens=word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [12]:

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred=pipeline.predict(X_test)

# display results
labels = np.unique(y_pred)
#confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
accuracy = (y_pred == y_test).mean()

print("Labels:", labels)
#print("Confusion Matrix:\n", confusion_mat)
print("Accuracy:", accuracy)

Labels: [0 1 2]
Accuracy: related                   0.785932
request                   0.886482
offer                     0.997711
aid_related               0.736344
medical_help              0.921880
medical_products          0.955752
search_and_rescue         0.974519
security                  0.980165
military                  0.967196
child_alone               1.000000
water                     0.950717
food                      0.931340
shelter                   0.927373
clothing                  0.986421
money                     0.978639
missing_people            0.986421
refugees                  0.967501
death                     0.961092
other_aid                 0.871987
infrastructure_related    0.931797
transport                 0.958651
buildings                 0.950717
electricity               0.979249
tools                     0.994355
hospitals                 0.990082
shops                     0.995117
aid_centers               0.987641
other_infrastructure      0.9

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
i=-1
for column in y_test.columns:
    i=i+1
    print (classification_report(y_test[column], y_pred[:,i], labels= labels))

             precision    recall  f1-score   support

          0       0.61      0.32      0.42      1530
          1       0.81      0.94      0.87      4976
          2       0.80      0.08      0.15        48

avg / total       0.76      0.79      0.76      6554

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5426
          1       0.81      0.39      0.53      1128
          2       0.00      0.00      0.00         0

avg / total       0.87      0.88      0.86      6554

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34
          2       0.00      0.00      0.00         0

avg / total       0.99      0.99      0.99      6554

             precision    recall  f1-score   support

          0       0.72      0.88      0.79      3851
          1       0.75      0.50      0.60      2703
          2       0.00      0.00    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
# specify parameters for grid search
parameters = {
    "clf__estimator__min_samples_split":[2,3],
    "clf__estimator__min_samples_leaf": [1,2]

}

# create grid search object
cv = GridSearchCV(estimator=pipeline,param_grid=parameters)

{'clf__estimator__min_samples_leaf': 1, 'clf__estimator__min_samples_split': 2}

In [ ]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
    
cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
i=-1
for column in y_test.columns:
    i=i+1
    print (classification_report(y_test[column], y_pred[:,i], labels= labels))

             precision    recall  f1-score   support

          0       0.64      0.33      0.44      1530
          1       0.82      0.94      0.87      4976
          2       0.75      0.12      0.21        48

avg / total       0.77      0.79      0.77      6554

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5426
          1       0.84      0.40      0.54      1128
          2       0.00      0.00      0.00         0

avg / total       0.88      0.88      0.86      6554

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34
          2       0.00      0.00      0.00         0

avg / total       0.99      0.99      0.99      6554

             precision    recall  f1-score   support

          0       0.72      0.87      0.79      3851
          1       0.73      0.52      0.61      2703
          2       0.00      0.00    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [5]:
import pickle
filename = 'classifier.pkl'
pickle.dump(cv, open(filename, 'wb'))

NameError: name 'cv' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.